In [1]:
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape

import tensorflow_datasets as tfds

In [57]:
# Thor DataLoad
# Specify the path to the manually downloaded dataset
data_dir = r'C:\Users\thorp\OneDrive\Dokumenter\Uni\Kandidat\Anvendt maskinlæring\Exam\Data\patch_camelyon'

# Load PatchCamelyon dataset using TFDS
def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, image # Here i make sure that labels are not passed through since the AE does not use labels

In [58]:
ds1,ds2,ds3 = tfds.load('patch_camelyon',split=['train[:20%]','test[:5%]','validation[:5%]'],
                        data_dir = data_dir,
                        download=False,
                        shuffle_files=True)

In [59]:
train_dataset       = ds1.map(convert_sample).batch(32)
validation_dataset  = ds3.map(convert_sample).batch(32)
test_dataset        = ds2.map(convert_sample).batch(32)

In [60]:
latent_dim = 2
# Encoder
encoder_inputs = tf.keras.layers.Input(shape=(96, 96, 3))
x = tf.keras.layers.Conv2D(32, 3, 2, activation='relu')(encoder_inputs)
x = tf.keras.layers.Conv2D(64, 3, 2, activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
encoded = tf.keras.layers.Dense(2 * latent_dim)(x)  # 2 for mean and standard deviation

encoder = tf.keras.models.Model(inputs=encoder_inputs, outputs=encoded)
encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 47, 47, 32)        896       
                                                                 
 conv2d_17 (Conv2D)          (None, 23, 23, 64)        18496     
                                                                 
 flatten_8 (Flatten)         (None, 33856)             0         
                                                                 
 dense_15 (Dense)            (None, 4)                 135428    
                                                                 
Total params: 154,820
Trainable params: 154,820
Non-trainable params: 0
_________________________________________________________________


In [61]:
decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=6*6*64, activation='relu', input_shape=(latent_dim,)),
    tf.keras.layers.Reshape(target_shape=(6, 6, 64)),  # To get in "image format"
    tf.keras.layers.Conv2DTranspose(128, 3, 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(64, 3, 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(32, 3, 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(3, 3, 2, padding='same', activation='sigmoid'),  # Adjust stride to get to 96x96x3
])

decoder.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 2304)              6912      
                                                                 
 reshape_8 (Reshape)         (None, 6, 6, 64)          0         
                                                                 
 conv2d_transpose_30 (Conv2D  (None, 12, 12, 128)      73856     
 Transpose)                                                      
                                                                 
 conv2d_transpose_31 (Conv2D  (None, 24, 24, 64)       73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_32 (Conv2D  (None, 48, 48, 32)       18464     
 Transpose)                                                      
                                                     

In [62]:
class VAE(tf.keras.Model):
    def __init__(self, latent_dim, encoder, decoder):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = encoder
        self.decoder = decoder
    
    def encode(self, x):
        params = self.encoder(x)
        return tf.split(params, num_or_size_splits=2, axis=1) # mean, logvar
        
    def decode(self, z):
        return self.decoder(z)
    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * 0.5) + mean #sigma= sqrt(exp(logvar))
    
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return tf.sigmoid(self.decode(eps))

In [63]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    vals = -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi)
    return tf.reduce_sum(vals, axis=raxis)

def compute_loss(model, x):
    # Output from encoder
    mean, logvar = model.encode(x)    
    # The reparameterization trick
    z = model.reparameterize(mean, logvar)    
    # We assume that p(x|z) is multivariate Bernoulli, ie. the final dense layer 
    # has a sigmoid activation function, see page. 11
    # in Kingma, D. P., & Welling, M. (2013).
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, 
                                                        labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])    
    # Assume normaility of p(z)
    logpz = log_normal_pdf(z, 0., 0.)    
    # Assume normality of q(z|x)
    logqz_x = log_normal_pdf(z, mean, logvar)
    # -tf.reduce_mean(decoder + sampler - encoder)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

In [64]:
@tf.function
def train_step(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [65]:
model = VAE(latent_dim, encoder, decoder)
optimizer = tf.keras.optimizers.Adam(1e-4)
test_sample = next(iter(test_dataset.take(1)))[:16]

def generate_and_save_images(model, epoch, test_sample):
    mean, logvar = model.encode(test_sample)
    z = model.reparameterize(mean, logvar)
    predictions = model.sample(z)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0], cmap='gray')
        plt.axis('off')

    plt.savefig('./vae-img/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [67]:
for epoch in range(1):
    for train_x, _ in train_dataset:
        train_step(model, train_x, optimizer)

    loss = tf.keras.metrics.Mean()
    for test_x in test_dataset:
        loss(compute_loss(model, test_x))
    variational_lower_bound = -loss.result()

    #print(f'Epoch: {epoch}, Test set variational lower bound:
   #        {variational_lower_bound}')
    generate_and_save_images(model, epoch, test_sample)

ValueError: Layer "model_1" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor: shape=(32, 96, 96, 3), dtype=float32, numpy=
array([[[[0.57254905, 0.3019608 , 0.6666667 ],
         [0.69411767, 0.41960788, 0.76470596],
         [0.81568635, 0.5529412 , 0.86274517],
         ...,
         [0.1137255 , 0.07843138, 0.39607847],
         [0.52156866, 0.33333334, 0.73333335],
         [0.427451  , 0.10196079, 0.5647059 ]],

        [[0.6       , 0.34509805, 0.65882355],
         [0.6784314 , 0.43529415, 0.7294118 ],
         [0.90196085, 0.6862745 , 0.95294124],
         ...,
         [0.15686275, 0.1254902 , 0.45098042],
         [0.27058825, 0.10980393, 0.50980395],
         [0.43137258, 0.16470589, 0.60784316]],

        [[0.7294118 , 0.49411768, 0.74509805],
         [0.7960785 , 0.6       , 0.8313726 ],
         [0.8313726 , 0.6784314 , 0.90196085],
         ...,
         [0.07843138, 0.03921569, 0.3803922 ],
         [0.3921569 , 0.27450982, 0.65882355],

In [68]:
print(train_x.shape)

(13, 96, 96, 3)
